In [0]:
# Mount google drive for loading and saving files 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Setting up tensorboard
LOG_DIR = 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_log'
get_ipython().system_raw('tensorboard --logdir {} --host 127.0.0.1 --port 6007 &'.format(LOG_DIR))
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://d3efa040.ngrok.io


In [0]:
# Importing all necessary libraries and defining constants
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn
import tensorboard
import os
import time
import math
import numpy as np
import glob
import sys
import re
tf.set_random_seed(0)
ALPHASIZE = 93
INTERNALSIZE = 512
NLAYERS = 3


In [0]:
# Provided by Martin Görner
def convert_from_alphabet(a):
    """Encode a character
    :param a: one character
    :return: the encoded value
    """
    if a == 9:
        return 1
    if a == 10:
        return 127 - 30  # LF
    elif 32 <= a <= 126:
        return a - 30
    else:
        return 0  # unknown


# encoded values:
# unknown = 0
# tab = 1
# space = 2
# all chars from 32 to 126 = c-30
# LF mapped to 127-30
def convert_to_alphabet(c, avoid_tab_and_lf=False):
    """Decode a code point
    :param c: code point
    :param avoid_tab_and_lf: if True, tab and line feed characters are replaced by '\'
    :return: decoded character
    """
    if c == 1:
        return 32 if avoid_tab_and_lf else 9  # space instead of TAB
    if c == 127 - 30:
        return 92 if avoid_tab_and_lf else 10  # \ instead of LF
    if 32 <= c + 30 <= 126:
        return c + 30
    else:
        return 0  # unknown


def encode_text(s):
    """Encode a string.
    :param s: a text string
    :return: encoded list of code points
    """
    s = s.replace('ü','ue').replace('ä','ae').replace('ö','oe')
    return list(map(lambda a: convert_from_alphabet(ord(a)), s))


def decode_to_text(c, avoid_tab_and_lf=False):
    """Decode an encoded string.
    :param c: encoded list of code points
    :param avoid_tab_and_lf: if True, tab and line feed characters are replaced by '\'
    :return:
    """
    return "".join(map(lambda a: chr(convert_to_alphabet(a, avoid_tab_and_lf)), c))


def sample_from_probabilities(probabilities, topn=ALPHASIZE):
    """Roll the dice to produce a random integer in the [0..ALPHASIZE] range,
    according to the provided probabilities. If topn is specified, only the
    topn highest probabilities are taken into account.
    :param probabilities: a list of size ALPHASIZE with individual probabilities
    :param topn: the number of highest probabilities to consider. Defaults to all of them.
    :return: a random integer
    """
    p = np.squeeze(probabilities)
    p[np.argsort(p)[:-topn]] = 0
    p = p / np.sum(p)
    return np.random.choice(ALPHASIZE, 1, p=p)[0]


def rnn_minibatch_sequencer(raw_data, batch_size, sequence_size, nb_epochs):
    """
    Divides the data into batches of sequences so that all the sequences in one batch
    continue in the next batch. This is a generator that will keep returning batches
    until the input data has been seen nb_epochs times. Sequences are continued even
    between epochs, apart from one, the one corresponding to the end of raw_data.
    The remainder at the end of raw_data that does not fit in an full batch is ignored.
    :param raw_data: the training text
    :param batch_size: the size of a training minibatch
    :param sequence_size: the unroll size of the RNN
    :param nb_epochs: number of epochs to train on
    :return:
        x: one batch of training sequences
        y: on batch of target sequences, i.e. training sequences shifted by 1
        epoch: the current epoch number (starting at 0)
    """
    data = np.array(raw_data)
    data_len = data.shape[0]
    # using (data_len-1) because we must provide for the sequence shifted by 1 too
    nb_batches = (data_len - 1) // (batch_size * sequence_size)
    assert nb_batches > 0, "Not enough data, even for a single batch. Try using a smaller batch_size."
    rounded_data_len = nb_batches * batch_size * sequence_size
    xdata = np.reshape(data[0:rounded_data_len], [batch_size, nb_batches * sequence_size])
    ydata = np.reshape(data[1:rounded_data_len + 1], [batch_size, nb_batches * sequence_size])

    for epoch in range(nb_epochs):
        for batch in range(nb_batches):
            x = xdata[:, batch * sequence_size:(batch + 1) * sequence_size]
            y = ydata[:, batch * sequence_size:(batch + 1) * sequence_size]
            x = np.roll(x, -epoch, axis=0)  # to continue the text from epoch to epoch (do not reset rnn state!)
            y = np.roll(y, -epoch, axis=0)
            yield x, y, epoch


def find_book(index, bookranges):
    return next(
        book["name"] for book in bookranges if (book["start"] <= index < book["end"]))


def find_book_index(index, bookranges):
    return next(
        i for i, book in enumerate(bookranges) if (book["start"] <= index < book["end"]))


def print_learning_learned_comparison(X, Y, losses, bookranges, batch_loss, batch_accuracy, epoch_size, index, epoch):
    """Display utility for printing learning statistics"""
    print()
    # epoch_size in number of batches
    batch_size = X.shape[0]  # batch_size in number of sequences
    sequence_len = X.shape[1]  # sequence_len in number of characters
    start_index_in_epoch = index % (epoch_size * batch_size * sequence_len)
    for k in range(batch_size):
      if k % 50 == 0:
        index_in_epoch = index % (epoch_size * batch_size * sequence_len)
        decx = decode_to_text(X[k], avoid_tab_and_lf=True)
        decy = decode_to_text(Y[k], avoid_tab_and_lf=True)
        bookname = find_book(index_in_epoch, bookranges)
        formatted_bookname = "{: <10.40}".format(bookname)  # min 10 and max 40 chars
        epoch_string = "{:4d}".format(index) + " (epoch {}) ".format(epoch)
        loss_string = "loss: {:.5f}".format(losses[k])
        print_string = epoch_string + formatted_bookname + " │ {} │ {} │ {}"
        print(print_string.format(decx, decy, loss_string))
        index += sequence_len
    # box formatting characters:
    # │ \u2502
    # ─ \u2500
    # └ \u2514
    # ┘ \u2518
    # ┴ \u2534
    # ┌ \u250C
    # ┐ \u2510
    format_string = "└{:─^" + str(len(epoch_string)) + "}"
    format_string += "{:─^" + str(len(formatted_bookname)) + "}"
    format_string += "┴{:─^" + str(len(decx) + 2) + "}"
    format_string += "┴{:─^" + str(len(decy) + 2) + "}"
    format_string += "┴{:─^" + str(len(loss_string)) + "}┘"
    footer = format_string.format('INDEX', 'BOOK NAME', 'TRAINING SEQUENCE', 'PREDICTED SEQUENCE', 'LOSS')
    print(footer)
    # print statistics
    batch_index = start_index_in_epoch // (batch_size * sequence_len)
    batch_string = "batch {}/{} in epoch {},".format(batch_index, epoch_size, epoch)
    stats = "{: <28} batch loss: {:.5f}, batch accuracy: {:.5f}".format(batch_string, batch_loss, batch_accuracy)
    print()
    print("TRAINING STATS: {}".format(stats))


class Progress:
    """Text mode progress bar.
    Usage:
            p = Progress(30)
            p.step()
            p.step()
            p.step(start=True) # to restart form 0%
    The progress bar displays a new header at each restart."""
    def __init__(self, maxi, size=100, msg=""):
        """
        :param maxi: the number of steps required to reach 100%
        :param size: the number of characters taken on the screen by the progress bar
        :param msg: the message displayed in the header of the progress bat
        """
        self.maxi = maxi
        self.p = self.__start_progress(maxi)()  # () to get the iterator from the generator
        self.header_printed = False
        self.msg = msg
        self.size = size

    def step(self, reset=False):
        if reset:
            self.__init__(self.maxi, self.size, self.msg)
        if not self.header_printed:
            self.__print_header()
        next(self.p)

    def __print_header(self):
        print()
        format_string = "0%{: ^" + str(self.size - 6) + "}100%"
        print(format_string.format(self.msg))
        self.header_printed = True

    def __start_progress(self, maxi):
        def print_progress():
            # Bresenham's algorithm. Yields the number of dots printed.
            # This will always print 100 dots in max invocations.
            dx = maxi
            dy = self.size
            d = dy - dx
            for x in range(maxi):
                k = 0
                while d >= 0:
                    print('=', end="", flush=True)
                    k += 1
                    d -= dx
                d += dy
                yield k

        return print_progress

def clean_text_from_emojies(shaketext):
  emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       u"\U0001f926-\U0001f937"
                       u"\u200d"
                       u"\u2640-\u2642" 
                       "]+", flags=re.UNICODE)
  return emoji_pattern.sub('',shaketext.read())

def read_data_files(directory,validation=True):
    """Read data files according to the specified glob pattern
    Optionnaly set aside the last file as validation data.
    No validation data is returned if there are 5 files or less.
    :param directory: for example "data/*.txt"
    :param validation: if True (default), sets the last file aside as validation data
    :return: training data, validation data, list of loaded file names with ranges
     If validation is
    """
    codetext = []
    bookranges = []
    shakelist = glob.glob(directory, recursive=True)
    print(shakelist)
    p = Progress(len(shakelist))
    for shakefile in shakelist:
        p.step()
        try:
          shaketext = open(shakefile, "r")
          start = len(codetext)
          #codetext.extend(encode_text(shaketext.read()))
          codetext.extend(encode_text(clean_text_from_emojies(shaketext)))
          end = len(codetext)
          bookranges.append({"start": start, "end": end, "name": shakefile.rsplit("/", 1)[-1]})
          shaketext.close()
        except:
          continue
    if len(bookranges) == 0:
        sys.exit("No training data has been found. Aborting.")

    # For validation, use roughly 90K of text,
    # but no more than 10% of the entire text
    # and no more than 1 book in 5 => no validation at all for 5 files or fewer.

    # 10% of the text is how many files ?
    total_len = len(codetext)
    validation_len = 0
    nb_books1 = 0
    for book in reversed(bookranges):
        validation_len += book["end"]-book["start"]
        nb_books1 += 1
        if validation_len > total_len // 10:
            break

    # 90K of text is how many books ?
    validation_len = 0
    nb_books2 = 0
    for book in reversed(bookranges):
        validation_len += book["end"]-book["start"]
        nb_books2 += 1
        if validation_len > 90*1024:
            break

    # 20% of the books is how many books ?
    nb_books3 = len(bookranges) // 5

    # pick the smallest
    nb_books = min(nb_books1, nb_books2, nb_books3)
    
    if nb_books == 0:
      cutoff = len(codetext) - int(len(codetext)/5)
    else:
      cutoff = bookranges[-nb_books]["start"]
    valitext = codetext[cutoff:]
    codetext = codetext[:cutoff]
    return codetext, valitext, bookranges


def print_data_stats(datalen, valilen, epoch_size):
    datalen_mb = datalen/1024.0/1024.0
    valilen_kb = valilen/1024.0
    print("\nTraining text size is {:.2f}MB with {:.2f}KB set aside for validation.".format(datalen_mb, valilen_kb)
          + " There will be {} batches per epoch".format(epoch_size))


def print_validation_header(validation_start, bookranges):
    bookindex = find_book_index(validation_start, bookranges)
    books = ''
    for i in range(bookindex, len(bookranges)):
        books += bookranges[i]["name"]
        if i < len(bookranges)-1:
            books += ", "
    print("{: <60}".format("\nValidating on " + books), flush=True)


def print_validation_stats(loss, accuracy):
    print("\nVALIDATION STATS:                                  loss: {:.5f},       accuracy: {:.5f}".format(loss,
                                                                                                           accuracy))


def print_text_generation_header():
    print()
    print("┌{:─^111}┐".format('Generating random text from learned state'))


def print_text_generation_footer():
    print()
    print("└{:─^111}┘".format('End of generation'))


def frequency_limiter(n, multiple=1, modulo=0):
    def limit(i):
        return i % (multiple * n) == modulo*multiple
    return limit

In [0]:
# Training RNN
def run_training_procedure(modelname, file_dir ,model_dir,log_dir ,seq_len=50):
  tf.reset_default_graph()
  SEQLEN = seq_len
  BATCHSIZE = 200
  learning_rate = 0.001  # fixed learning rate
  dropout_pkeep = 0.8    # some dropout

  # load data, either shakespeare, or the Python source of Tensorflow itself
  shakedir = file_dir+'/*.txt'
  codetext, valitext, bookranges = read_data_files(shakedir,validation=True)
  # display some stats on the data
  epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
  print_data_stats(len(codetext), len(valitext), epoch_size)

  #
  # the model (see FAQ in README.md)
  #
  lr = tf.placeholder(tf.float32, name='lr')  # learning rate
  pkeep = tf.placeholder(tf.float32, name='pkeep')  # dropout parameter
  batchsize = tf.placeholder(tf.int32, name='batchsize')

  # inputs
  X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
  Xo = tf.one_hot(X, ALPHASIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
  # expected outputs = same sequence shifted by 1 since we are trying to predict the next character
  Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
  Yo_ = tf.one_hot(Y_, ALPHASIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
  # input state
  Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

  # using a NLAYERS=3 layers of GRU cells, unrolled SEQLEN=30 times
  # dynamic_rnn infers SEQLEN from the size of the inputs Xo

  # How to properly apply dropout in RNNs: see README.md
  cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]
  # "naive dropout" implementation
  dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=pkeep) for cell in cells]
  multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
  multicell = rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)  # dropout for the softmax layer

  Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)
  # Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
  # H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence

  H = tf.identity(H, name='H')  # just to give it a name

  # Softmax layer implementation:
  # Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
  # then apply softmax readout layer. This way, the weights and biases are shared across unrolled time steps.
  # From the readout point of view, a value coming from a sequence time step or a minibatch item is the same thing.

  Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
  Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
  Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
  loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
  Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
  Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
  Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]
  train_step = tf.train.AdamOptimizer(lr).minimize(loss)

  # stats for display
  seqloss = tf.reduce_mean(loss, 1)
  batchloss = tf.reduce_mean(seqloss)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))
  loss_summary = tf.summary.scalar("batch_loss", batchloss)
  acc_summary = tf.summary.scalar("batch_accuracy", accuracy)
  summaries = tf.summary.merge([loss_summary, acc_summary])

  # Init Tensorboard stuff. This will save Tensorboard information into a different
  # folder at each run named 'log/<timestamp>/'. Two sets of data are saved so that
  # you can compare training and validation curves visually in Tensorboard.
  timestamp = str(math.trunc(time.time()))
  summary_writer = tf.summary.FileWriter(log_dir+"/" + timestamp + "-training")
  validation_writer = tf.summary.FileWriter(log_dir+"/" + timestamp + "-validation")

  # Init for saving models. They will be saved into a directory named 'checkpoints'.
  # Only the last checkpoint is kept.
  saver = tf.train.Saver(max_to_keep=10)

  # for display: init the progress bar
  DISPLAY_FREQ = 50
  _50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
  progress = Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

  # init
  istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
  init = tf.global_variables_initializer()
  sess = tf.Session()
  sess.run(init)
  step = 0

  # training loop
  for x, y_, epoch in rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=10):

      # train on one minibatch
      feed_dict = {X: x, Y_: y_, Hin: istate, lr: learning_rate, pkeep: dropout_pkeep, batchsize: BATCHSIZE}
      _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

      # log training data for Tensorboard display a mini-batch of sequences (every 50 batches)
      if step  % _50_BATCHES == 0:
          feed_dict = {X: x, Y_: y_, Hin: istate, pkeep: 1.0, batchsize: BATCHSIZE}  # no dropout for validation
          y, l, bl, acc, smm = sess.run([Y, seqloss, batchloss, accuracy, summaries], feed_dict=feed_dict)
          print_learning_learned_comparison(x, y, l, bookranges, bl, acc, epoch_size, step, epoch)
          summary_writer.add_summary(smm, step)

      # run a validation step every 50 batches
      # The validation text should be a single sequence but that's too slow (1s per 1024 chars!),
      # so we cut it up and batch the pieces (slightly inaccurate)
      # tested: validating with 5K sequences instead of 1K is only slightly more accurate, but a lot slower.
      if step % _50_BATCHES == 0 and len(valitext) > 0:
          VALI_SEQLEN = 1*1024  # Sequence length for validation. State will be wrong at the start of each sequence.
          bsize = len(valitext) // VALI_SEQLEN
          print_validation_header(len(codetext), bookranges)
          vali_x, vali_y, _ = next(rnn_minibatch_sequencer(valitext, bsize, VALI_SEQLEN, 1))  # all data in 1 batch
          vali_nullstate = np.zeros([bsize, INTERNALSIZE*NLAYERS])
          feed_dict = {X: vali_x, Y_: vali_y, Hin: vali_nullstate, pkeep: 1.0,  # no dropout for validation
                       batchsize: bsize}
          ls, acc, smm = sess.run([batchloss, accuracy, summaries], feed_dict=feed_dict)
          print_validation_stats(ls, acc)
          # save validation data for Tensorboard
          validation_writer.add_summary(smm, step)

      # display a short text generated with the current weights and biases (every 150 batches)
      if step // 6 % _50_BATCHES == 0:
          print_text_generation_header()
          ry = np.array([[convert_from_alphabet(ord("K"))]])
          rh = np.zeros([1, INTERNALSIZE * NLAYERS])
          for k in range(100):
              ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
              rc = sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
              print(chr(convert_to_alphabet(rc)), end="")
              ry = np.array([[rc]])
          print_text_generation_footer()

      # save a checkpoint (every 500 batches)
      if step // 10 % _50_BATCHES == 0:
          saved_file = saver.save(sess, model_dir + '/rnn_train_' + timestamp, global_step=step)
          print("Saved file: " + saved_file)

      # display progress bar
      progress.step(reset=step % _50_BATCHES == 0)

      # loop state around
      istate = ostate
      step += BATCHSIZE * SEQLEN

  return saver.save(sess, model_dir + '/final_rnn_train_' + modelname, global_step=step)

In [0]:
# Using the generated model to generate text
def generate_sample(topn=10,author=''):

  # use topn=10 for all but the last one which works with topn=2 for Shakespeare and topn=3 for Python
  ncnt = 0
  output_string = ''
  with tf.Session() as sess:
      new_saver = tf.train.import_meta_graph(author+'.meta')
      new_saver.restore(sess, author)
      x = convert_from_alphabet(ord("L"))
      x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

      # initial values
      y = x
      h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
      for i in range(10000):
          yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})

          # If sampling is be done from the topn most likely characters, the generated text
          # is more credible and more "english". If topn is not set, it defaults to the full
          # distribution (ALPHASIZE)

          # Recommended: topn = 10 for intermediate checkpoints, topn=2 or 3 for fully trained checkpoints

          c = sample_from_probabilities(yo, topn=topn)
          y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
          c = chr(convert_to_alphabet(c))
          output_string+=c
  return output_string

In [0]:
# Train model
model_path = 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models'
file_path='drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus'
log_path ='drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_log'
model_name = run_training_procedure('chats',model_dir=model_path,file_dir = file_path,log_dir = log_path,seq_len=200)

['drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_papa.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_rafi.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_paula.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_mama.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_saunsch.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_vera.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/sofa_chat.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_familie.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_soziologie.txt', 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_corpus/chat_felix.t

In [0]:
# Generate a series of documents with different topn parameters
sample_docs = 'drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/sample_docs'
for i in [2,3,5,8,10,15,20,30,50,100,0]:
  text_ = generate_sample(topn=i,author=model_name)
  text_ = text_.replace('[','\n[')
  with open(sample_docs+'/sample_'+str(i)+'.txt','w+') as outp:
    outp.write(text_)

INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models/final_rnn_train_chats-40800000
INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models/final_rnn_train_chats-40800000
INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models/final_rnn_train_chats-40800000
INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models/final_rnn_train_chats-40800000
INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models/final_rnn_train_chats-40800000
INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tensorflow-rnn-shakespeare-master/miniproject/chat_models/final_rnn_train_chats-40800000
INFO:tensorflow:Restoring parameters from drive/My Drive/NNandNLP/tens